<a href="https://colab.research.google.com/github/Mechanics-Mechatronics-and-Robotics/PytorchBasics/blob/main/08_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#import matplotlib.pyplot as plt

## Initialization

In [2]:
bc = datasets.load_breast_cancer()

X, Y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)

#scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test  = torch.from_numpy(X_test.astype(np.float32))
Y_train = torch.from_numpy(Y_train.astype(np.float32))
Y_test = torch.from_numpy(Y_test.astype(np.float32))

Y_train = Y_train.view(Y_train.shape[0], 1)
Y_test = Y_test.view(Y_test.shape[0], 1)


## Model prediction

In [3]:
input_size = n_features
output_size = 1

#The first approach
#model = nn.Linear(input_size, output_size)

#The second approach
class LogisticRegression(nn.Module):

  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    #define layers
    self.linear = nn.Linear(n_input_features, 1)

  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted

model = LogisticRegression(input_size)

## Loss that is MSE (mean square error)

In [4]:
loss = nn.BCELoss()


# Training

In [7]:
lr = 0.01
n = 100

optimizer = torch.optim.SGD(model.parameters(), lr=lr)

for epoch in range(n):
  #prediciton
  y_pred = model(X_train)
  #loss
  L = loss(y_pred, Y_train)
  #gradients
  L.backward()
  #update
  optimizer.step()
  #zero gradients
  optimizer.zero_grad()

  if (epoch+1) % 10 == 0:
    print(f'epoch {epoch+1}: loss = {L.item():.4f}')

with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(Y_test).sum()/float(Y_test.shape[0])
  print(f'accuracy = {acc:.4f}')

epoch 10: loss = 0.2230
epoch 20: loss = 0.2148
epoch 30: loss = 0.2075
epoch 40: loss = 0.2009
epoch 50: loss = 0.1948
epoch 60: loss = 0.1893
epoch 70: loss = 0.1843
epoch 80: loss = 0.1796
epoch 90: loss = 0.1753
epoch 100: loss = 0.1713
accuracy = 0.9123


## Visualization

In [6]:
# plt.plot(X_numpy, Y_numpy, 'ro')
# plt.plot(X_numpy, predicted, 'b')